In [ ]:
import os

: 

In [ ]:
%pwd

: 

In [8]:
os.chdir("../")

In [9]:
%pwd

'd:\\Desktop\\Final_Projects'

In [10]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    max_seq_length: int
    dtype: str
    load_in_4bit: bool
    model_name: str
    r: int
    target_modules: list
    lora_alpha: int
    lora_dropout: int
    bias: str
    use_gradient_checkpointing: bool
    random_state: int

In [7]:
from transformers import AutoTokenizer
import os
from DocQnA.logging import logger

In [ ]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
import torch
from transformers import TrainingArguments
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    max_seq_length: int
    dtype: str
    load_in_4bit: bool
    model_name: str
    r: int
    target_modules: list
    lora_alpha: int
    lora_dropout: int
    bias: str
    use_gradient_checkpointing: bool
    random_state: int

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name=self.config.model_name,
            max_seq_length=self.config.max_seq_length,
            dtype=self.config.dtype,
            load_in_4bit=self.config.load_in_4bit
        )

        model = FastLanguageModel.get_peft_model(
            model,
            r=self.config.r,
            target_modules=self.config.target_modules,
            lora_alpha=self.config.lora_alpha,
            lora_dropout=self.config.lora_dropout,
            bias=self.config.bias,
            use_gradient_checkpointing=self.config.use_gradient_checkpointing,
            random_state=self.config.random_state,
            max_seq_length=self.config.max_seq_length
        )

        trainer = SFTTrainer(
            model=model,
            train_dataset=dataset,
            dataset_text_field="text",
            max_seq_length=self.config.max_seq_length,
            args=TrainingArguments(
                per_device_train_batch_size=32,
                gradient_accumulation_steps=1,
                warmup_steps=5,
                max_steps=50,
                learning_rate=2e-4,
                fp16=not torch.cuda.is_bf16_supported(),
                bf16=torch.cuda.is_bf16_supported(),
                logging_steps=2,
                optim="adamw_8bit",
                weight_decay=0.01,
                lr_scheduler_type="linear",
                seed=self.config.random_state,
                output_dir="outputs"
            )
        )

        trainer_stats = trainer.train()

try:
    config = ModelTrainerConfig(
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
        model_name="unsloth/zephyr-sft-bnb-4bit",
        r=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        lora_alpha=32,
        lora_dropout=0,
        bias="none",
        use_gradient_checkpointing=True,
        random_state=3407
    )
    model_trainer = ModelTrainer(config=config)
    model_trainer.train()
except Exception as e:
    raise e